In [1]:
using Random, Statistics, LinearAlgebra, CSV, DataFrames, LsqFit, Plots

# Constants
const N = 3
const pi = 3.141592654
const v0 = 0.03
const tmax = 10
const start = 2

# Simulation Parameters
radii = range(1.0, 14.0, length=100) # 100 points between 1 and 14
L = 10.0 # Length of the box

# DataFrame to store critical exponents results
exponents = DataFrame(radius = Float64[], beta = Float64[], gamma = Float64[])

for ro in radii
    println("Processing radius: $ro")

    # Initialization of position and velocity arrays
    x = rand(N) * L
    y = rand(N) * L
    theta = rand(N) * 2 * pi
    vx = v0 .* cos.(theta)
    vy = v0 .* sin.(theta)

    # DataFrame to store simulation results for this radius
    results = DataFrame(r = Float64[], vm = Float64[], cum = Float64[], susc = Float64[])

    for r in 0.1:0.1:3.0
        r1 = r / 2.0
        for t in 1:tmax
            x .+= vx
            y .+= vy
            x .= mod.(x, L)
            y .= mod.(y, L)

            vvx = zeros(N)
            vvy = zeros(N)
            for i in 1:N
                s = 0.0
                c = 0.0
                for j in 1:N
                    dx = min(abs(x[i] - x[j]), L - abs(x[i] - x[j]))
                    dy = min(abs(y[i] - y[j]), L - abs(y[i] - y[j]))
                    d = sqrt(dx^2 + dy^2)
                    if d <= ro
                        s += vy[j]
                        c += vx[j]
                    end
                end
                dir = atan(s, c)
                noise = (2 * rand() - 1) * pi * r1
                vvx[i] = v0 * cos(dir + noise)
                vvy[i] = v0 * sin(dir + noise)
            end
            vx .= vvx
            vy .= vvy

            if t > start
                vmx = sum(vx)
                vmy = sum(vy)
                vm = sqrt(vmx^2 + vmy^2) / (N * v0)
                cum = 1.0 - (vm^4 / (3 * vm^2 * vm^2))
                susc = L^2 * abs(vm^2 - vm * vm)
                push!(results, (r, vm, cum, susc))
            end
        end
    end

    # Fit the data to extract critical exponents
    r_c = 2.0  # Hypothetical critical noise value

    # Order parameter fitting function
    order_model(r, β) = abs(r .- r_c).^β
    order_fit = curve_fit(order_model, results.r, results.vm, [0.5])
    β = order_fit.param[1]

    # Susceptibility fitting function
    susc_model(r, γ) = abs(r .- r_c).^(-γ)
    susc_fit = curve_fit(susc_model, results.r, results.susc, [0.5])
    γ = susc_fit.param[1]

    # Store the critical exponents for this radius
    push!(exponents, (ro, β, γ))
end

# Save the exponents to a CSV file
CSV.write("vicsek_critical_exponents.csv", exponents)

# Load data
data = CSV.read("vicsek_critical_exponents.csv", DataFrame)

# Plotting critical exponents
plot(data.radius, data.beta, label="β (Order Parameter Exponent)", xlabel="Interaction Radius (R)", ylabel="Critical Exponents", title="Critical Exponents vs Interaction Radius")
plot!(data.radius, data.gamma, label="γ (Susceptibility Exponent)")

savefig("vicsek_critical_exponents_vs_radius.pdf")


Processing radius: 1.0


LoadError: MethodError: no method matching abs(::Vector{Float64})

[0mClosest candidates are:
[0m  abs([91m::Union{Float16, Float32, Float64}[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mfloat.jl:609[24m[39m
[0m  abs([91m::Unsigned[39m)
[0m[90m   @[39m [90mBase[39m [90m[4mint.jl:187[24m[39m
[0m  abs([91m::T[39m) where T<:Dates.Period
[0m[90m   @[39m [32mDates[39m [90mC:\Users\fatem\AppData\Local\Programs\Julia-1.9.3\share\julia\stdlib\v1.9\Dates\src\[39m[90m[4mperiods.jl:103[24m[39m
[0m  ...


In [ ]:
using Random, Statistics, LinearAlgebra, CSV, DataFrames, LsqFit, Plots

# Constants
const N = 100
const pi = 3.141592654
const v0 = 0.03
const tmax = 100
const start = 50

# Simulation Parameters
radii = range(2.0, 10.0, length=100) # 100 points between 1 and 14
L = 10.0 # Length of the box

# DataFrame to store critical exponents results
exponents = DataFrame(radius = Float64[], beta = Float64[], gamma = Float64[])

for ro in radii
    println("Processing radius: $ro")

    # Initialization of position and velocity arrays
    x = rand(N) * L
    y = rand(N) * L
    theta = rand(N) * 2 * pi
    vx = v0 .* cos.(theta)
    vy = v0 .* sin.(theta)

    # DataFrame to store simulation results for this radius
    results = DataFrame(r = Float64[], vm = Float64[], cum = Float64[], susc = Float64[])

    for r in 0.1:0.1:3.0
        r1 = r / 2.0
        for t in 1:tmax
            x .+= vx
            y .+= vy
            x .= mod.(x, L)
            y .= mod.(y, L)

            vvx = zeros(N)
            vvy = zeros(N)
            for i in 1:N
                s = 0.0
                c = 0.0
                for j in 1:N
                    dx = min(abs(x[i] - x[j]), L - abs(x[i] - x[j]))
                    dy = min(abs(y[i] - y[j]), L - abs(y[i] - y[j]))
                    d = sqrt(dx^2 + dy^2)
                    if d <= ro
                        s += vy[j]
                        c += vx[j]
                    end
                end
                dir = atan(s, c)
                noise = (2 * rand() - 1) * pi * r1
                vvx[i] = v0 * cos(dir + noise)
                vvy[i] = v0 * sin(dir + noise)
            end
            vx .= vvx
            vy .= vvy

            if t > start
                vmx = sum(vx)
                vmy = sum(vy)
                vm = sqrt(vmx^2 + vmy^2) / (N * v0)
                cum = 1.0 - (vm^4 / (3 * vm^2 * vm^2))
                susc = L^2 * abs(vm^2 - vm * vm)
                push!(results, (r, vm, cum, susc))
            end
        end
    end

    # Fit the data to extract critical exponents
    r_c = 2.0  # Hypothetical critical noise value

    # Order parameter fitting function
    order_model(x, p) = abs.(x .- r_c).^p[1]
    order_fit = curve_fit(order_model, results.r, results.vm, [0.5])
    β = order_fit.param[1]

    # Susceptibility fitting function
    susc_model(x, p) = abs.(x .- r_c).^(-p[1])
    susc_fit = curve_fit(susc_model, results.r, results.susc, [0.5])
    γ = susc_fit.param[1]

    # Store the critical exponents for this radius
    push!(exponents, (ro, β, γ))
end

# Save the exponents to a CSV file
CSV.write("vicsek_critical_exponents.csv", exponents)

# Load data
data = CSV.read("vicsek_critical_exponents.csv", DataFrame)

# Plotting critical exponents
plot(data.radius, data.beta, label="β (Order Parameter Exponent)", xlabel="Interaction Radius (R)", ylabel="Critical Exponents", title="Critical Exponents vs Interaction Radius")
plot!(data.radius, data.gamma, label="γ (Susceptibility Exponent)")

savefig("vicsek_critical_exponents_vs_radius2.pdf")


Processing radius: 2.0


Processing radius: 2.080808080808081

In [4]:

savefig("vicsek_critical_exponents_vs_radius.pdf")


"C:\\Users\\fatem\\Downloads\\BC_Thesis\\vicsek_critical_exponents_vs_radius.pdf"